<a href="https://colab.research.google.com/github/tenihasina/AffectiveReaction/blob/master/LLM_Tiime_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tiime scientific coding test

Alice and Bob play the following game:

- Alice chooses a 10-letter word from a fixed dictionary, uniformly at random
- At each turn Bob proposes a word from the same dictionary
- For each proposal, Alice returns the "score" of the proposed word: each correct letter gives 1 point, plus an additional point if it is at the right position.
- The total score of Bob is the sum of scores over k rounds with k fixed to 5, 10 or 15.

The goal is to maximize the score of Bob over 100 games  - where each game lasts k rounds - (or its expectation). The score is then normalized so that the maximum score is 100.

You are allowed to use reasonable precomputation (less than one hour) and your code should play any game in a reasonable amount of time (less than a few minutes). Running time will also be considered in grading. You can use any library, even if it is not installed. To install a library with `pip`, just paste the command with `!` before: `!pip install numpy` (this one is already installed).

Create a copy of this notebook that you will share. Runtime > Run all should execute your code without error. If you have suggestions for improvements, feel free to add them and we might take them into account in the evaluation.

Please send your score for each size along with the link to your solution notebook.

## Testing code

Execute the code below to download the word list and define the testing class.
It is recommended to read this code.

**DO NOT MODIFY ANYTHING IN THIS SECTION**

In [9]:
!wget -nc https://gematria.tech/words_10.txt 2> /dev/null

In [10]:
from collections import Counter

def score(word1, word2):
    """scoring function
    """
    assert len(word1) == len(word2)
    ans = 0
    hist = Counter(word1)
    for c in word2:
        if hist[c]:
            hist[c] -= 1
            ans += 1
    for c1, c2 in zip(word1, word2):
        ans += c1 == c2
    return ans

assert score("abc", "abd") == 4
assert score("abc", "bca") == 3

In [11]:
import random
from tqdm.autonotebook import tqdm

class Simulator:
    def __init__(self, path="words_10.txt"):
        with open(path) as f:
            self.words = list(map(str.strip, f))
            self.words_set = self.words

    def play(self, bob, rounds):
        total_score = 0
        word = random.choice(self.words)
        bob.new_game()
        for _ in range(rounds):
            guess = bob.guess()
            if guess not in self.words_set:
                return 0
            s = score(word, guess)
            bob.clue(s)
            total_score += s
        return total_score

    def score(self, bob, games, rounds):
        ans = sum(self.play(bob, rounds) / 20 / rounds for _ in tqdm(range(games)))
        return 100 * ans / games


In [12]:
class ExampleBob:
    """
    You should copy and modify this code to implement your solution
    """
    def __init__(self, words, rounds):
        """
        Call this only once to create your player. Any precomputation should go here.
        """
        self.words = words
        self.rounds = rounds

    def new_game(self):
        """
        Prepare for a new game
        """
        pass

    def guess(self):
        """
        Make a new guess
        """
        return random.choice(self.words)

    def clue(self, score):
        """
        The last guess got this score.
        """
        pass


In [13]:
from time import time

simulator = Simulator()

for rounds in [5, 10, 15]:
    t = time()
    bob = ExampleBob(simulator.words.copy(), rounds)
    print('%.02f s' % (time() - t))
    print("score", simulator.score(bob, games=100, rounds=rounds))


0.00 s


  0%|          | 0/100 [00:00<?, ?it/s]

score 24.76
0.00 s


  0%|          | 0/100 [00:00<?, ?it/s]

score 24.394999999999992
0.00 s


  0%|          | 0/100 [00:00<?, ?it/s]

score 23.546666666666667


## Your code

This is the only section you should modify. You can add any number of cells. Comments are welcome but not necessary.

In [48]:
import numpy as np

class Bob:
    """
    You should only modify this code
    """
    def __init__(self, words, rounds):
        """
        Call this only once to create your player. Any precomputation should go here.
        """
        self.words = words
        self.rounds = rounds
        self.possible_answers = dict()
        self.turn = 0

        cnt_per_letter = Counter()
        for cnt in [Counter(word) for word in self.words]:
            cnt_per_letter.update(cnt)
        most_common_letters = ''.join([pair[0] for pair in cnt_per_letter.most_common(10)])
        filtered_words = self.words
        for letter in most_common_letters:
            temp = [word for word in filtered_words if letter in word]
            if len(temp) > 0:
              filtered_words = temp
        best_words = dict()
        for guess in filtered_words:
            comparison = [(word, score(word, guess)) for word in simulator.words]
            best_words[guess] = np.mean([v for k,v in comparison])
        sorted_dict_desc = dict(sorted(best_words.items(), key=lambda item: item[1], reverse=True))

        self.initial_guess = next(iter(sorted_dict_desc))


    def new_game(self):
        """
        Prepare for a new game
        """
        self.answer = self.initial_guess
        self.possible_answers = dict()
        self.turn = 0

    def guess(self):
        """
        Make a new guess
        """
        if self.turn < 1:
            self.answer = self.initial_guess
            self.possible_answers[0] = self.words
        else:
            word_vs_answer = dict()
            words = self.possible_answers[self.turn-1]
            for word in words:
              if score(word, self.answer) in word_vs_answer:
                word_vs_answer[int(score(word, self.answer))].append(word)
              else:
                word_vs_answer[int(score(word, self.answer))] = [word]
            closest_score = min(word_vs_answer.keys(), key=lambda k: abs(k - self.result))
            self.possible_answers[self.turn] = word_vs_answer[closest_score]
            self.answer = random.choice(self.possible_answers[self.turn])
        self.turn += 1
        return self.answer

    def clue(self, score):
        """
        The last guess got this score.
        """
        self.result = score

# Evaluation

**DO NOT MODIFY**

In [52]:
from time import time

simulator = Simulator()

for rounds in [5, 10, 15]:
    t = time()
    bob = Bob(simulator.words.copy(), rounds)
    print('%.02f s' % (time() - t))
    print("score", simulator.score(bob, games=100, rounds=rounds))


3.23 s


  0%|          | 0/100 [00:00<?, ?it/s]

score 29.84999999999998
3.18 s


  0%|          | 0/100 [00:00<?, ?it/s]

score 59.489999999999974
3.69 s


  0%|          | 0/100 [00:00<?, ?it/s]

score 73.42333333333332
